In [1]:
import sys
sys.path.append('/home/disk/p/jkcm/Code')
from tools.LoopTimer import LoopTimer

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

In [3]:
# Hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = r'/home/disk/eos4/jkcm/Data/MNISTData'
MODEL_STORE_PATH = r'/home/disk/p/jkcm/Projects/MNISTmodel'

In [4]:
# # transforms to apply to the data
# trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# # MNIST dataset
# train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=False)
# test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

RuntimeError: Dataset not found. You can use download=True to download it

In [5]:
# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:


model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
lt = LoopTimer(num_epochs*len(train_loader))
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        lt.update()
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))


3.33%      ETA 17:27:17      time left: 13.3 minutes       Epoch [1/5], Step [100/600], Loss: 0.2374, Accuracy: 91.00%
6.67%      ETA 17:27:19      time left: 12.9 minutes       Epoch [1/5], Step [200/600], Loss: 0.0907, Accuracy: 97.00%
10.00%      ETA 17:27:21      time left: 12.5 minutes       Epoch [1/5], Step [300/600], Loss: 0.2435, Accuracy: 92.00%
13.33%      ETA 17:27:20      time left: 12.0 minutes       Epoch [1/5], Step [400/600], Loss: 0.1354, Accuracy: 97.00%
16.67%      ETA 17:27:21      time left: 11.5 minutes       Epoch [1/5], Step [500/600], Loss: 0.0660, Accuracy: 99.00%
20.00%      ETA 17:27:21      time left: 11.1 minutes       Epoch [1/5], Step [600/600], Loss: 0.0867, Accuracy: 97.00%
23.33%      ETA 17:27:22      time left: 10.6 minutes       Epoch [2/5], Step [100/600], Loss: 0.0575, Accuracy: 98.00%
26.67%      ETA 17:27:25      time left: 10.2 minutes       Epoch [2/5], Step [200/600], Loss: 0.1328, Accuracy: 96.00%
30.00%      ETA 17:27:28      time left: 9

In [11]:

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

Test Accuracy of the model on the 10000 test images: 96.17 %


/home/disk/p/jkcm/anaconda3/lib/python3.5/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 103), ('y', 102)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
